# Config

In [90]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues
from ffpackage.viz import compareFranchises

In [71]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 1
weekEnd = 2
extra_defensive_points = 18
pr_bonus = 1
kr_bonus = 3
username = 'dirtywizard'

# Extract

In [72]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [73]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [74]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [75]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)

# Transform

In [76]:
# Customize predictions for league
# multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + extra_defensive_points

# Adjust point calculation based on PR/KR
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Calculate relative points
preds['pts_proj'] = preds['pts_proj'].fillna(0)
preds = preds.sort_values(by='pts_proj', ascending=False, ignore_index=True)
for pos in rostLims.columns[1:]:
    cond1 = preds['position']==pos
    floor = preds.loc[cond1, 'pts_proj'].reset_index(drop=True)[rostLims.loc[rostLims['username']==username, pos][0]]
    preds.loc[cond1, 'rel_proj'] = preds.loc[cond1, 'pts_proj'] - floor

# Reduce
preds = preds[['week', 'player_id', 'pts_proj', 'rel_proj', 'full_name', 'age', 'years_exp', 'weight', 'height',
       'team', 'position', 'fantasy_positions', 'depth_chart_order',
       'depth_chart_position', 'status', 'injury_status', 'injury_notes',
       'injury_start_date', 'practice_participation', 'practice_description','active'
       ]]

In [77]:
# Merge
# Merge in points
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

# Waiver Report

In [85]:
# Waiver report
# Sum points across the season
totals_proj = full.groupby('player_id')['pts_proj'].sum().reset_index()
totals_rel = full.groupby('player_id')['rel_proj'].sum().reset_index()
unis = full.drop_duplicates(subset='player_id', ignore_index=True).drop(columns=['pts_proj', 'rel_proj'])
waivers = unis.merge(totals_proj, how='left', on='player_id').merge(totals_rel, how='left', on='player_id')
# Select only players on user's roster or unclaimed, and who would improve user's points
minpts = waivers.loc[(waivers['display_name']==username)&(waivers['taxi'].isna())]['pts_proj'].min()
cond1 = waivers['display_name']==username
cond2 = waivers['display_name'].isna()
cond3 = waivers['pts_proj']>=minpts
waivers = waivers.loc[(cond1|cond2)&cond3].reset_index(drop=True)
# Reduce
waivers = waivers[[
    'week', 'player_id', 'pts_proj', 'rel_proj', 'franchise_name', 'full_name', 'age', 'years_exp', 'weight',
       'height', 'team', 'position', 'taxi', 'starters', 'players', 'fantasy_positions', 'depth_chart_order',
       'depth_chart_position', 'status', 'injury_status', 'injury_notes',
       'injury_start_date', 'practice_participation', 'practice_description','active'
]]

# Visualize Total Points

In [91]:
full.columns

Index(['week', 'player_id', 'pts_proj', 'rel_proj', 'full_name', 'age',
       'years_exp', 'weight', 'height', 'team', 'position',
       'fantasy_positions', 'depth_chart_order', 'depth_chart_position',
       'status', 'injury_status', 'injury_notes', 'injury_start_date',
       'practice_participation', 'practice_description', 'active', 'owner_id',
       'taxi', 'starters', 'players', 'user_id', 'franchise_name',
       'display_name'],
      dtype='object')

In [ ]:
compareFranchises(full, )

### Left Off Here:
##### Build trade report
##### RosterOptimizer
##### Viz franchise points

# Write to CSV

In [ ]:
# sums.to_csv(f'processed_{date_string}.csv', index=False)
# waivers.to_csv(f'waivers_{date_string}.csv', index=False)

In [87]:
# Starter report
full.loc[full['starters']==True].groupby('franchise_name')['rel_proj'].sum().sort_values()

franchise_name
Comeback 👑’s               -84.5002
2014champ                  -35.7518
Idk much about soccer      -28.8230
Count of Monte Christian   -25.8082
DangeRUSS Last Ride        -16.3294
GusTheBus                  -14.4044
📜 Providence 🪬 Spirits 🥂   -13.4572
kevinbash                    1.7472
Verdanks Vacqueros FFC       7.6152
Pretty Big Wieners           8.0010
Saskatoon Squatches         16.4104
Croccity Body Snatchers     37.7902
Name: rel_proj, dtype: float64

In [88]:
full.loc[full['starters']==True].groupby('franchise_name')['pts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          19.2950
Comeback 👑’s                226.8142
kevinbash                   240.8168
Idk much about soccer       255.7966
2014champ                   261.7978
Count of Monte Christian    271.7414
GusTheBus                   286.9252
📜 Providence 🪬 Spirits 🥂    299.7472
Verdanks Vacqueros FFC      307.0548
Saskatoon Squatches         308.7100
Pretty Big Wieners          309.3306
Croccity Body Snatchers     339.1198
Name: pts_proj, dtype: float64